# Create Masternuts and Hexbins

In [ ]:
import geopandas as gpd
import h3
from shapely.geometry import Polygon, Point
import pandas as pd
from pathlib import Path
from timezonefinder import TimezoneFinder
import consts
import overpy
import json

In [ ]:
tf:TimezoneFinder = TimezoneFinder()

## Create Masternuts

In [ ]:
# Import Nuts EU
gpd_nuts_eu = gpd.read_file(consts.PATH_NUTS_EU)
gpd_nuts_eu = gpd_nuts_eu[gpd_nuts_eu.LEVL_CODE == 3]

# Remove some countries
gpd_nuts_eu = gpd_nuts_eu[~gpd_nuts_eu.CNTR_CODE.isin(['TR'])]

# Import Nuts UK
gpd_nuts_uk = gpd.read_file(consts.PATH_NUTS_UK)
gpd_nuts_uk.rename(columns={
    'nuts318cd': 'NUTS_ID',
    'nuts318nm': 'NAME_LATN'
    }, inplace=True)
gpd_nuts_uk.to_crs(gpd_nuts_eu.crs, inplace=True)

# Import Bosnia. Has no NUTS
gdf_bosnia = gpd.read_file(consts.PATH_BORDERS)
gdf_bosnia = gdf_bosnia[gdf_bosnia.CNTR_ID == 'BA']
gdf_bosnia.rename(columns={'ISO3_CODE': 'NUTS_ID', 'NAME_ENGL': 'NAME_LATN'}, inplace=True)
gdf_bosnia = gdf_bosnia[['NUTS_ID', 'NAME_LATN', 'geometry']]

# Merge
gpd_nuts = pd.concat([gpd_nuts_eu, gpd_nuts_uk, gdf_bosnia], ignore_index=True)
gpd_nuts = gpd_nuts[['NUTS_ID', 'NAME_LATN', 'geometry']]

# Add timezone information to the GeoDataFrame
gpd_nuts['timezone'] = gpd_nuts.apply(
    lambda row: tf.timezone_at(lat=row.geometry.centroid.y, lng=row.geometry.centroid.x), axis=1
)

# Drop overseas
gdf_boundaries = gpd.read_file(consts.PATH_BOUNDARIES)
gpd_nuts = gpd.overlay(gpd_nuts, gdf_boundaries, how='intersection')

# Save masternuts
gpd_nuts.to_file(consts.PATH_MASTERNUTS)

## Create Grid

In [ ]:
# Define the bounding box for Europe (approximate)
min_lat, min_lon = -25.0,34.0
max_lat, max_lon = 45.0,72.0

# Define the resolution for the hexagons (adjust as needed)
resolution = 4  # This resolution corresponds to hexagons with an edge length of ~50 km

# Generate hexagons within the bounding box
hexagons = []
for lat in range(int(min_lat * 100), int(max_lat * 100), 1):
    for lon in range(int(min_lon * 100), int(max_lon * 100), 1):
        hex_id = h3.latlng_to_cell(lat / 100.0, lon / 100.0, resolution)
        hexagons.append(hex_id)

# Convert hexagons to polygons
hexagon_polygons = [Polygon(h3.cell_to_boundary(h)) for h in set(hexagons)]

# Create a GeoDataFrame
gdf_hexagons = gpd.GeoDataFrame({'geometry': hexagon_polygons})

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf_hexagons.crs = 'EPSG:4326'

## Cut Grid

In [ ]:
df_eu = gpd.read_file(Path('../data/countries-europe.gpkg'))

# Remove countries
df_eu = df_eu[~df_eu.ISO_A2_EH.isin(['RU', 'BY', 'UA'])]

df_eu = df_eu.dissolve()

In [ ]:
# Cut and export
gdf_hexagons_cut = gpd.overlay(gdf_hexagons, df_eu, how='intersection')
gdf_hexagons_cut = gdf_hexagons_cut[['geometry']].copy()

# Add Timezone
gdf_hexagons_cut['timezone'] = gdf_hexagons_cut.apply(
    lambda row: tf.timezone_at(lat=row.geometry.centroid.y, lng=row.geometry.centroid.x), axis=1
)

gdf_hexagons_cut.reset_index(drop=False, inplace=True)
gdf_hexagons_cut.rename(columns={'index':'NUTS_ID'}, inplace=True)
gdf_hexagons_cut.to_file(consts.PATH_HEXAGON)

## Download Labels from OSM

In [26]:
gdf_hexagons_cut = gpd.read_file(consts.PATH_HEXAGON)

array([-25.4042823 ,  34.81500886,  31.56952478,  71.1803653 ])

In [ ]:
api = overpy.Overpass()

def get_features(place):

    x1 = gdf_hexagons_cut.total_bounds[1]
    y1 = gdf_hexagons_cut.total_bounds[0]
    x2 = gdf_hexagons_cut.total_bounds[3]
    y2 = gdf_hexagons_cut.total_bounds[2]

    # Call Overpass
    result = api.query(f"""
        (
        node["place"="{place}"]({x1},{y1},{x2},{y2});
        );
        (._;>;);
        out body;
    """)

    # Create GeoJSON Features
    features = []
    for n in result.nodes:
        features.append({
        "type": "Feature",
        "properties": n.tags,
        "geometry": {
            "type": "Point",
            "coordinates": [
                float(n.lon),
                float(n.lat)
            ]
        }
    })

    return features

geojson = {
  "type": "FeatureCollection",
  "features": get_features('city') + get_features('town')
}

# Export
json.dump(geojson, open(Path('../export/citylabels.geojson'), 'w'), ensure_ascii=False, indent = 2)    